In [ ]:
import numpy as np

import scipy as sp
import scipy.stats as stats
from scipy.stats import linregress,t
from scipy.optimize import curve_fit

import matplotlib.pyplot as plt
from matplotlib import rc
plt.rc('text', usetex=True)
font = {'family' : 'serif',
        'size'   : 14}
plt.rc('font', **font)
plt.rc('ytick', labelsize=28) 
plt.rc('xtick', labelsize=28)
from matplotlib.colors import LinearSegmentedColormap

import json
import os
import math


from ipywidgets import interact,interactive
import ipywidgets as widgets

from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

In [ ]:
times = [-15,-10,-5,2,10,20,30,40]


In [ ]:
def ReadFiles(Dir):
    Accept = []
    for d in os.listdir(Dir):
        try:
            if('Synapse_a.json' in os.listdir(Dir+d) ):
                Accept.append(d)
        except:
            pass
    print(Accept)   
    Syn_a_arr = []
    bg_l      = []
    for a in Accept:
        with open(Dir+a+'/Synapse_a.json', 'r') as fp: Syn_a_arr.append(json.load(fp))
    Areas_arr = []
    Dist_arr  = []
    bg_arr = []
    kk = 0
    for Syn_a in Syn_a_arr:
        kk+=1
        areas = np.array([S["area"] for S in Syn_a])
        if(not Syn_a[0]["Times"]==times):
            for l in np.sort(list(set(times)-set(Syn_a[0]["Times"]))[::-1]):
                areas = np.insert(areas,times.index(l),math.nan,-1)
                
        dist_a = [S["distance"] for S in Syn_a]
        Dist_arr  = Dist_arr+dist_a
        Areas_arr.append(areas)
    Areas_arr_b = np.vstack(Areas_arr)
    Dist_arr_b = np.array(Dist_arr)#    
    Areas_arr_b = (Areas_arr_b.T/Areas_arr_b[:,:3].mean(axis=1)).T
    return Dist_arr_b,Areas_arr_b,(Areas_arr,Dist_arr)

#===================================================================================
#===================================================================================
def barplot_annotate_brackets(num1, num2, data, center, height, yerr=None, dh=.05, barh=.05, fs=20, maxasterix=1,offS=0.6):
    """ 
    Annotate barplot with p-values.

    :param num1: number of left bar to put bracket over
    :param num2: number of right bar to put bracket over
    :param data: string to write or number for generating asterixes
    :param center: centers of all bars (like plt.bar() input)
    :param height: heights of all bars (like plt.bar() input)
    :param yerr: yerrs of all bars (like plt.bar() input)
    :param dh: height offset over bar / bar + yerr in axes coordinates (0 to 1)
    :param barh: bar height in axes coordinates (0 to 1)
    :param fs: font size
    :param maxasterix: maximum number of asterixes to write (for very small p-values)
    """

    if type(data) is str:
        text = data
    else:
        # * is p < 0.05
        # ** is p < 0.005
        # *** is p < 0.0005
        # etc.
        text = ''
        p = .05

        while data < p:
            text += '*'
            p /= 10.

            if maxasterix and len(text) == maxasterix:
                break

        if(p<0.05):
            text = r'*'
        else:
            text = r'n.s.'
    lx, ly = center[num1], height[num1]
    rx, ry = center[num2], height[num2]

    if yerr:
        ly += yerr[num1]
        ry += yerr[num2]

    ax_y0, ax_y1 = plt.gca().get_ylim()
    dh *= (ax_y1 - ax_y0)
    barh *= (ax_y1 - ax_y0)

    y = max(ly, ry) + dh

    barx = [lx, lx, rx, rx]
    bary = [y, y+barh, y+barh, y]
    mid = ((lx+rx)/2, y+barh)

    plt.plot(barx, bary, c='black')

    kwargs = dict(ha='center', va='bottom')
    if fs is not None:
        kwargs['fontsize'] = fs

    plt.text(*mid, text, **kwargs)

    return max(ly, ry)

def pltDist(S):
    A=S.T
    fig,ax = plt.subplots(1,3,figsize=(15,5))
    x  = ax[0].hist(A[:,0],bins=30,density=True)
    ax[0].axvline(np.nanmean(A[:,0]),c='g')
    ax[0].plot((x[1][:-1]+x[1][1:])/2,x[0]/(sum(x[0])*(x[1][1]-x[1][0])),'r')
    means = []
    sem = []
    for i,a in enumerate(A.T[:-1]):
        x = np.histogram(a,bins=15,density=True)
        if(i==0):
            ax[1].plot((x[1][:-1]+x[1][1:])/2,x[0]/(sum(x[0])*(x[1][1]-x[1][0])),'r')
        else:
            ax[1].plot((x[1][:-1]+x[1][1:])/2,x[0]/(sum(x[0])*(x[1][1]-x[1][0])),'b')
        means.append(a.mean())
        sem.append(stats.sem(a)) 
    ax[2].errorbar(x = range(len(means)),y = means, yerr=sem, label='both limits (default)')
    ax[2].set_ylim(np.nanmin(A),np.nanmax(A))
    plt.show()
    print('means:',means)
    
def pltDyn(S):
    A = S
    col ='k'
    for i in range(2):
        fig,ax = plt.subplots(1,3,figsize=(15,5))
        x = ax[0].hist(A[i+1]-A[i])
        x = ax[1].scatter((A[i+2]-A[i+1]),(A[i+1]-A[i]),c=col)
        col = np.where(A[i+1]-A[i]<0,'r',np.where(A[i+1]-A[i]>0,'b','r'))
        ax[1].set_ylabel('\huge{$i$}')
        ax[1].set_xlabel('\huge{$i+\delta i$}')
        ax[1].set_xticks([])
        ax[1].set_yticks([])
        ax[1].axvline(0,c='r')
        ax[1].axhline(0,c='r')
        x = ax[2].hist(A[i+2]-A[i+1])
        plt.show()
        ss = sum(np.logical_and((A[i+2]-A[i+1])<0,(A[i+1]-A[i])<0))
        gg = sum(np.logical_and((A[i+2]-A[i+1])>0,(A[i+1]-A[i])>0))
        gs = sum(np.logical_and((A[i+2]-A[i+1])>0,(A[i+1]-A[i])<0))
        sg = sum(np.logical_and((A[i+2]-A[i+1])<0,(A[i+1]-A[i])>0))
        g = sum((A[i+1]-A[i])>0)
        s = sum((A[i+1]-A[i])<0)
        print("Correlation: {:0.2}".format(np.corrcoef(A[i+2]-A[i+1],A[i+1]-A[i+0])[1,0]))
        print("shrink-shrink: {:2.2%}".format(ss/len(A[i+2]-A[i+1])))
        print("grow-grow: {:2.2%}".format(gg/len(A[i+2]-A[i+1])))
        print("grow-shrink: {:2.2%}".format(gs/len(A[i+2]-A[i+1])))
        print("shrink-grow: {:2.2%}".format(sg/len(A[i+2]-A[i+1])))
        print("shrink given that you grew: {:2.2%}".format(sg/g))
        print("grow given that you shrunk: {:2.2%}".format(gs/s))
        print('='*100)

In [ ]:
BaseDir = 'Data/'
Dir = BaseDir+'CA1_15Spine/Sham/'
D15,A1,(A1raw,_) = ReadFiles(Dir)
A1raw15 = np.vstack(A1raw)
Dir = BaseDir+'CA1_7Spine/Sham/'
D7,A1,(A1raw,_) = ReadFiles(Dir)
A1raw7 = np.vstack(A1raw)
Dir = BaseDir+'CA1_3Spine/Sham/'
D3,A1,(A1raw,_) = ReadFiles(Dir)
A1raw3 = np.vstack(A1raw)
Dir = BaseDir+'CA1_1Spine/Sham/'
D1,A1,(A1raw,_) = ReadFiles(Dir)
A1raw1 = np.vstack(A1raw)
Araw = np.vstack([A1raw1,A1raw3,A1raw7,A1raw15]).T
Araw = Araw[:,~np.isnan(Araw.sum(axis=0))]

# Figures

## Figure 1

### c

In [ ]:
A=Araw[~np.isnan(Araw.sum(axis=1)),:]
means_expt = []
sem_expt = []
red = np.array([1,0,0])
blue = np.array([0,0,1])
for i,a in enumerate(A):
    x = np.histogram(a,bins=15,density=True)
    sns.kdeplot(a,c=i/7*blue + (7-i)/7*red,lw=3)
    means_expt.append(a.mean())
    sem_expt.append(stats.sem(a))
plt.ylabel('')
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
ax.set_yticks([])
plt.show()

In [ ]:
plt.errorbar(x = np.arange(8),y = means_expt, yerr=sem_expt,lw=4,c='r',label='Pop. mean')
plt.plot(np.arange(8),Araw[:,11],lw=4,c='k',alpha=0.25)
plt.plot(np.arange(8),Araw[:,2],lw=4,c='k',alpha=0.25)
plt.plot(np.arange(8),Araw[:,0],lw=4,c='k',alpha=0.25,label='Ex. Spines')

plt.yticks(fontsize=48)
plt.xticks(fontsize=48)
leg = plt.legend(fontsize=54,loc=1,bbox_to_anchor=(1.3,1.65))
leg.get_frame().set_linewidth(0.0)
ax  = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)

### d

In [ ]:
col ='k'
A = Araw.T[~np.isnan(Araw.T.sum(axis=1)),:].T
Del = []
for i in range(7):
    Del.append(A[i+1]-A[i])
fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(111, projection='3d')
nbins = 50
red = np.array([1,0,0])
blue = np.array([0,0,1])
for i,(z,d) in enumerate(zip([120,100,80,60, 40, 20, 0],Del)):
    hist, bins = np.histogram(d, bins=nbins,density=True)
    xs = (bins[:-1] + bins[1:])/2

    ax.bar(xs, hist, zs=z, zdir='x', 
           color=i/7*blue + (7-i)/7*red,alpha=0.5,width=(xs[-1]-xs[0])/nbins)
    mu, std = stats.norm.fit(d)
    x = np.linspace(-0.6,0.6)
    p = stats.norm.pdf(x, mu, std)
    ax.plot(x,p,zs=z, zdir='x', linewidth=2,c='k')
ax.view_init(elev=30., azim=-50)
ax.set_ylim(-0.6,0.6)
ax.set_xticks([])
ax.set_zticks([])
ax.yaxis.labelpad = 10
ax.tick_params(axis='y', labelsize=40)

### e

In [ ]:
col ='k'
A = Araw.T[~np.isnan(Araw.T.sum(axis=1)),:].T
Del = []
for i in range(7):
    Del.append(A[i+1]-A[i])
DelF = np.vstack(Del).flatten()
mu = -0.0007436013701439731
std = 0.07383571332975278
x = np.linspace(DelF.min(),DelF.max(),100)
plt.hist(DelF,bins=100,width=0.013,density=True,color='r')
p = stats.norm.pdf(x, mu, std)
plt.plot(x, p, 'k', linewidth=2)
plt.xlim([-0.5,0.5])
plt.text(x=0.05,y=5,s="Mean: {:0.3}".format(mu),fontsize=24)
plt.text(x=0.1,y=4.3,s="STD: {:0.3}".format(std),fontsize=24)
plt.yticks([])
m = stats.lognorm.fit(DelF)
x = np.linspace(DelF.min(),DelF.max())
p = stats.lognorm.pdf(x, m[0], loc=m[1],scale=m[2])
stats.kstest(DelF,'lognorm',args=(m[0],m[1],m[2]))
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
plt.show()

### f

In [ ]:
colors = [(1.0,0.0, 0.0),(0,0,1)] # first color is black, last is red
cm = LinearSegmentedColormap.from_list(
        "Custom", colors, N=20)
A = Araw#.T
ArawD = A[1:] - A[:-1] 
CorrMat = np.zeros((7,7))
for i,a1 in enumerate(ArawD):
    for j,a2 in enumerate(ArawD):
        CorrMat[i,j] = np.corrcoef(a1[~np.logical_or(np.isnan(a1),np.isnan(a2))],a2[~np.logical_or(np.isnan(a1),np.isnan(a2))])[1,0]

for i in range(7):
    CorrMat[i,i] = math.nan
fig,ax = plt.subplots(figsize=(12,12))
g = ax.imshow(CorrMat,cmap=cm)
cbar = plt.colorbar(g,fraction=0.046, pad=0.04)
cbar.ax.tick_params(labelsize=64)
plt.xticks([0,1,2,3,4,5,6],np.arange(1,8),fontsize=64)
plt.yticks([0,1,2,3,4,5,6],np.arange(1,8),fontsize=64)
ax = plt.gca()
# Minor ticks
ax.set_xticks(np.arange(-.5, 7, 1), minor=True)
ax.set_yticks(np.arange(-.5, 7, 1), minor=True)

# Gridlines based on minor ticks
ax.grid(which='minor', color='w', linestyle='-', linewidth=2)

# Remove minor ticks
ax.tick_params(which='minor', bottom=False, left=False)

### g

In [ ]:
A = Araw
col ='k'
i=5
fig,ax = plt.subplots()
x = ax.scatter((A[i+2]-A[i+1]),(A[i+1]-A[i]),c='r')
slope, intercept, r_value, p_value, std_err = stats.linregress((A[i+2]-A[i+1]),(A[i+1]-A[i]))
amin = min((A[i+2]-A[i+1]).min(),(A[i+1]-A[i]).min())
amax = max((A[i+2]-A[i+1]).max(),(A[i+1]-A[i]).max())
plt.plot(np.arange(amin,amax,0.1),np.arange(amin,amax,0.1)*slope+intercept,'k',lw=3)

A1 = np.roll(Araw,-1,axis=0)[:-2,:]-Araw[:-2,:]
A2 = np.roll(Araw,-2,axis=0)[:-2,:]-np.roll(Araw,-1,axis=0)[:-2,:]
Corr = []
for a1,a2 in zip(A1,A2):
    i+=1
    Corr.append(np.corrcoef(a1,a2)[1,0])
plt.text(x=-0.15,y=0.39,s="Avg. Correlation: {:0.3}".format(np.mean(Corr)),fontsize=24)
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)



### h

In [ ]:
lowermean = []
lowerstd  = []
uppermean = []
upperstd  = []
Del_tot = []
for Cutoff in np.arange(0.2,0.8,0.15):
    col ='k'
    A = Araw
    Del1 = []
    for i in range(7):
        mask = np.logical_and(A[i]>Cutoff,A[i]<Cutoff+0.15)
        Del1.append(A[i+1,mask]-A[i,mask])
    Del_tot.append(Del1)
fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(111, projection='3d')
nbins = 50
red = np.array([1,0,0])
blue = np.array([0,0,1])
P = []
for i,(z,d) in enumerate(zip([0.2,0.35,0.5,0.65],Del_tot)):
    d = np.hstack(d)
    hist, bins = np.histogram(d, bins=50,density=True)
    xs = (bins[:-1] + bins[1:])/2
    m = stats.lognorm.fit(d)
    ax.bar(xs, hist, zs=z, zdir='x', color=red,alpha=0.5,width=(xs[-1]-xs[0])/nbins)
    x = np.linspace(-0.5,0.6)
    p1 = stats.lognorm.pdf(x, m[0], loc=m[1],scale=m[2])
    P.append(p1)
    ax.plot(x,p1,zs=z, zdir='x', linewidth=2,c='k')
ax.view_init(elev=30., azim=-30)
ax.set_ylim(-0.6,0.6)
plt.xticks(fontsize=40)
plt.yticks(fontsize=40)
ax.set_zticks([])
ax.yaxis.labelpad = 10
ax.xaxis.labelpad = 10
plt.gca().invert_xaxis()

## Figure 2

### a

In [ ]:
fig,ax = plt.subplots(figsize=(12,24))
col ='k'
A = Araw.T
Del = A
DelF = np.vstack(Del[:,0]).flatten()#np.vstack(Del).flatten()
DelF = np.ma.masked_invalid(DelF)
red   = np.array([1,0,0])
mu, std = stats.norm.fit(DelF)
x = np.linspace(DelF.min(),DelF.max())
plt.hist(DelF,bins=50,density=True,color='red',label='Expt. data')
p = stats.norm.pdf(x, mu, std)
k = stats.kstest(DelF,'norm',args=(mu,std))
plt.plot(x, p, c=blue, linewidth=5)#,label='Gaussian,\n $k = $ {:0.2} \n  $p$ = {:0.2}'.format(k.statistic,k.pvalue))
stats.kstest(DelF,'norm',args=(mu,std))
plt.plot([],[], marker='s',linestyle='None',c=blue, linewidth=5,
         label='Gaussian$^*$,\n $k = $ {:0.2}'.format(k.statistic,k.pvalue))

mu, std = stats.maxwell.fit(DelF)
p = stats.maxwell.pdf(x, mu, std)
k = stats.kstest(DelF,'maxwell',args=(mu,std))
plt.plot(x, p, c=0.8*blue, linewidth=5)#,label='Maxwell,\n $k = $ {:0.2} \n  $p$ = {:0.2}'.format(k.statistic,k.pvalue))
plt.plot([],[], marker='s',linestyle='None',c=0.8*blue, linewidth=5,
         label='Maxwell$^*$,\n $k = $ {:0.2}'.format(k.statistic,k.pvalue))

m = stats.gamma.fit(DelF)
p = stats.gamma.pdf(x, m[0], loc=m[1],scale=m[2])
k = stats.kstest(DelF,'gamma',args=(m[0],m[1],m[2]))
plt.plot(x, p, c=0.6*blue, linewidth=5)#,label='Gamma,\n $k = $ {:0.2} \n  $p$ = {:0.2}'.format(k.statistic,k.pvalue))
plt.plot([],[], marker='s',linestyle='None',c=0.6*blue, linewidth=5,
         label='Gamma,\n $k = $ {:0.2}'.format(k.statistic,k.pvalue))

m = stats.skewnorm.fit(DelF)
p = stats.skewnorm.pdf(x, m[0], loc=m[1],scale=m[2])
k = stats.kstest(DelF,'skewnorm',args=(m[0],m[1],m[2]))
plt.plot(x, p, c=0.4*blue, linewidth=5)#,label='Skew gaussian,\n $k = $ {:0.2} \n  $p$ = {:0.2}'.format(k.statistic,k.pvalue))
plt.plot([],[], marker='s',linestyle='None',c=0.4*blue, linewidth=5,
         label='Skew gaussian$^*$,\n $k = $ {:0.2}'.format(k.statistic,k.pvalue))

m = stats.lognorm.fit(DelF)
p = stats.lognorm.pdf(x, m[0], loc=m[1],scale=m[2])
k = stats.kstest(DelF,'lognorm',args=(m[0],m[1],m[2]))

plt.plot(x, p, 'k', linewidth=5,)
plt.plot([],[], 'sk', linewidth=5,
         label='LogNormal,\n $k = $ {:0.2}'.format(k.statistic,k.pvalue))
plt.yticks([])
plt.xticks(fontsize=64)
plt.xlim([0.2,1.5])
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
leg = ax.legend(loc=1,fontsize=48, markerscale=3,bbox_to_anchor=(1.1,1.))
leg.get_frame().set_linewidth(0.0)


### b

In [ ]:
plt.subplots
lowermean = []
lowerstd  = []
uppermean = []
upperstd  = []
semMean = []
semSTD  = []
for Cutoff in np.arange(0.2,1.0,0.15):
    col ='k'
    A = Araw
    Del1 = []
    for i in range(7):
        mask = np.logical_and(A[i]>Cutoff,A[i]<Cutoff+0.15)
        Del1.append(A[i+1,mask]-A[i,mask])
    DelF = np.hstack(Del1).flatten()
    DelF = np.hstack(Del1).flatten()
    mu, std = stats.norm.fit(DelF)
    semMean.append(stats.bootstrap((DelF,), np.mean, confidence_level=0.95,
                         random_state=1, method='percentile').standard_error)
    semSTD.append(stats.bootstrap((DelF,), np.std, confidence_level=0.95,
                     random_state=1, method='percentile').standard_error)
    lowermean.append(mu)
    lowerstd.append(std)
ml = stats.linregress(np.arange(0.2,1.0,0.15),lowermean)
sl = stats.linregress(np.arange(0.2,1.0,0.15),lowerstd)
plt.errorbar(np.arange(0.2,1.0,0.15),lowermean,semMean,capsize=5,ls='None',ms=15,fmt='.',color='r')
plt.plot(np.arange(0.2,1.0,0.15),np.arange(0.2,1.0,0.15)*ml.slope+ml.intercept,ls='--',c=blue+[0.5,0.5,0],lw=3,label='Best fit LN Model')
plt.errorbar(np.arange(0.2,1.0,0.15),lowerstd,semSTD,capsize=5,ls='None',ms=7,fmt='s',color='r')
plt.plot(np.arange(0.2,1.0,0.15),np.arange(0.2,1.0,0.15)*sl.slope+sl.intercept,ls='--',c=blue*0.5,lw=3)

print("Slope of mean: {:.3} and intercept: {:.3}".format(ml.slope,ml.intercept))
print("Slope of std: {:.3} and intercept: {:.3}".format(sl.slope,sl.intercept))
plt.text(x=0.3,y=0.1,s="Fit of std. of changes",rotation=13,fontsize=24)
plt.text(x=0.4,y=-0.14,s="Fit of mean changes",rotation=-18,fontsize=24)
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)

plt.xticks(fontsize=24)
plt.yticks(fontsize=24)
plt.show()

### c

In [ ]:
def ContLN():
    fmean = sp.interpolate.interp1d(np.arange(-5.0,5.0,0.15),np.arange(-5.0,5.0,0.15)*ml.slope+ml.intercept)
    fstd = sp.interpolate.interp1d(np.arange(-5.0,5.0,0.15),np.arange(-5.0,5.0,0.15)*sl.slope+sl.intercept)
    fstd2 = sp.interpolate.interp1d(np.arange(-5.0,5.0,0.15),np.arange(-5.0,5.0,0.15)*0.0933+0.0518)


    S = [Araw[0]]
    for i in range(1,8):
        sMean = fmean(S[-1])+1
        sStd  = fstd(S[-1])
        lmean = np.log(((sMean)**2)/np.sqrt(sStd**2+(sMean)**2))
        lsig = np.sqrt(np.log((sStd/(sMean))**2 + 1))
        yi = np.random.lognormal(lmean,lsig)
        S.append(S[-1]+yi-1)
    return np.array(S)
S_MC = []
for k in range(1000):
    S_MC.append(ContLN())
S_MC = np.array(S_MC)
X1 = [] 
X2 = []
for i,s in enumerate(S_MC[:,-1,:]):
    x = np.histogram(s,bins=15,density=True,range=(np.nanmin(S_MC), np.nanmax(Araw)))
    X1.append(x[0]/(sum(x[0])*(x[1][1]-x[1][0])))
    X2.append((x[1][:-1]+x[1][1:])/2)
X1 = np.array(X1)
X2 = np.array(X2)
S1 = [] 
S2 = []
for a in Araw:
    x = np.histogram(a,bins=15,density=True,range=(np.nanmin(Araw), np.nanmax(Araw)))
    S1.append(x[0]/(sum(x[0])*(x[1][1]-x[1][0])))
    S2.append((x[1][:-1]+x[1][1:])/2)
S1 = np.array(S1)
S2 = np.array(S2)
plt.fill_between(X2[0], X1.mean(axis=0)-X1.std(axis=0), X1.mean(axis=0)+X1.std(axis=0),
                 color='b',alpha=0.25)
plt.plot(S2[0], S1.mean(axis=0), 'r-',lw=3,label='Exp. data')

plt.fill_between(S2[0], S1.mean(axis=0)-S1.std(axis=0), S1.mean(axis=0)+S1.std(axis=0),
                 color='r',alpha=0.25)

for i,s in enumerate(S_MC[0,1:,:]):
    if(i==0):
        sns.kdeplot(s,c='k',alpha=0.5,
                label='E.g. evolution')
    else:
        sns.kdeplot(s,c='k',alpha=0.5)
plt.plot(X2[0], X1.mean(axis=0), 'b-',lw=3,label='Model pred.')
leg = plt.legend(fontsize=24)
leg.get_frame().set_linewidth(0.0)
plt.yticks([])
plt.xlim([0,1.8])
plt.ylabel('')
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
plt.show()

In [ ]:
plt.plot(np.mean(S_MC.mean(axis=-1),axis=0))
plt.errorbar(x = np.arange(8),y = np.mean(S_MC.mean(axis=-1),axis=0), 
             yerr=sp.stats.sem(S_MC[0],axis=1) ,lw=4,c='b')
plt.ylim([0.4,0.6])
plt.yticks(fontsize=48)
plt.xticks(fontsize=48)
plt.show()

### d

In [ ]:
A1 = np.roll(S_MC,-1,axis=-2)[:,:-2,:]-S_MC[:,:-2,:]
A2 = np.roll(S_MC,-2,axis=-2)[:,:-2,:]-np.roll(S_MC,-1,axis=-2)[:,:-2,:]
A1 = np.swapaxes(A1,-2,-1)
A2 = np.swapaxes(A2,-2,-1)
Corr = []
for a1,a2 in zip(A1,A2):
    for i in range(6):
        Corr.append(np.corrcoef(a1[:,i],a2[:,i])[1,0])

a1 = A1[0,np.logical_and(A1[0,:,0]!=0,A2[0,:,0]!=0),0]
a2 = A2[0,np.logical_and(A1[0,:,0]!=0,A2[0,:,0]!=0),0]
plt.scatter(A1[0,np.logical_and(A1[0,:,0]!=0,A2[0,:,0]!=0),0],
            A2[0,np.logical_and(A1[0,:,0]!=0,A2[0,:,0]!=0),0],c='b')
slope, intercept, r_value, p_value, std_err = stats.linregress(a1,a2)
amin = -0.3
amax = 0.26
plt.plot(np.arange(amin,amax,0.01),np.arange(amin,amax,0.01)*slope+intercept,'k',lw=3)

#x = np.corrcoef(A1,A2)
plt.text(x=-0.1,y=0.3,s="Avg. Correlation: {:0.3}".format(np.mean(Corr)),fontsize=24)
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
plt.show()

### e

In [ ]:
plt.subplots
lowermean = []
lowerstd  = []
uppermean = []
upperstd  = []
semMean = []
semSTD  = []
for Cutoff in np.arange(0.2,1.0,0.15):
    col ='k'
    A = Araw
    Del1 = []
    for i in range(7):
        mask = np.logical_and(A[i]>Cutoff,A[i]<Cutoff+0.15)
        Del1.append(A[i+1,mask]-A[i,mask])
    DelF = np.hstack(Del1).flatten()
    DelF = np.hstack(Del1).flatten()
    mu, std = stats.norm.fit(DelF)
    semMean.append(stats.bootstrap((DelF,), np.mean, confidence_level=0.95,
                         random_state=1, method='percentile').standard_error)
    semSTD.append(stats.bootstrap((DelF,), np.std, confidence_level=0.95,
                     random_state=1, method='percentile').standard_error)
    lowermean.append(mu)
    lowerstd.append(std)
ml = stats.linregress(np.arange(0.2,1.0,0.15),lowermean)
sl = stats.linregress(np.arange(0.2,1.0,0.15),lowerstd)
plt.errorbar(np.arange(0.2,1.0,0.15),lowermean,semMean,capsize=5,ls='None',ms=15,fmt='.',color='r')

plt.errorbar(np.arange(0.2,1.0,0.15),lowerstd,semSTD,capsize=5,ls='None',ms=7,fmt='s',color='r')

print("Slope of mean: {:.3} and intercept: {:.3}".format(ml.slope,ml.intercept))
print("Slope of std: {:.3} and intercept: {:.3}".format(sl.slope,sl.intercept))
plt.text(x=0.5,y=0.04,s="Fit of std. of changes",rotation=13,fontsize=24)
plt.text(x=0.55,y=-0.08,s="Fit of mean changes",rotation=-18,fontsize=24)

plt.plot(np.arange(0.2,1.0,0.15),np.arange(0.2,1.0,0.15)*ml.slope+1.2*ml.intercept,ls='dotted',c=blue+[0.5,0.5,0],lw=3,label='Alt. fit Model')
plt.plot(np.arange(0.2,1.0,0.15),np.arange(0.2,1.0,0.15)*sl.slope,ls='dotted',c=blue*0.5,lw=3) 
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
plt.xticks(fontsize=24)
plt.yticks(fontsize=24)
plt.show()


### f

In [ ]:
def ContLN2():
    fmean = sp.interpolate.interp1d(np.arange(-5.0,5.0,0.15),np.arange(-5.0,5.0,0.15)*ml.slope+1.2*ml.intercept)
    fstd = sp.interpolate.interp1d(np.arange(-5.0,5.0,0.15),np.arange(-5.0,5.0,0.15)*sl.slope)#e+0.4*sl.intercept)
    fstd2 = sp.interpolate.interp1d(np.arange(-5.0,5.0,0.15),np.arange(-5.0,5.0,0.15)*0.0933+0.0518)



    S = [Araw[0]]
    for i in range(1,8):
        sMean = fmean(S[-1]) + S[-1]
        sStd  = fstd(S[-1])
        lmean = np.log((sMean**2)/np.sqrt(sStd**2+sMean**2))
        lsig = np.sqrt(np.log((sStd/sMean)**2 + 1))
        yi = np.random.lognormal(lmean,lsig)
        S.append(yi+S[i-1]-S[-1])
    return np.array(S)

S_MC = []
for k in range(1000):
    S_MC.append(ContLN2())
S_MC = np.array(S_MC)
X1 = [] 
X2 = []
for i,s in enumerate(S_MC[:,-1,:]):
    x = np.histogram(s,bins=15,density=True,range=(np.nanmin(Araw), np.nanmax(Araw)))
    X1.append(x[0]/(sum(x[0])*(x[1][1]-x[1][0])))
    X2.append((x[1][:-1]+x[1][1:])/2)
X1 = np.array(X1)
X2 = np.array(X2)
S1 = [] 
S2 = []
for a in Araw:
    x = np.histogram(a,bins=15,density=True,range=(np.nanmin(Araw), np.nanmax(Araw)))
    S1.append(x[0]/(sum(x[0])*(x[1][1]-x[1][0])))
    S2.append((x[1][:-1]+x[1][1:])/2)
S1 = np.array(S1)
S2 = np.array(S2)
plt.fill_between(X2[0], X1.mean(axis=0)-X1.std(axis=0), X1.mean(axis=0)+X1.std(axis=0),
                 color='b',alpha=0.25)
plt.plot(S2[0], S1.mean(axis=0), 'r-',lw=3,label='Exp. data')

plt.fill_between(S2[0], S1.mean(axis=0)-S1.std(axis=0), S1.mean(axis=0)+S1.std(axis=0),
                 color='r',alpha=0.25)

for i,s in enumerate(S_MC[0,1:,:]):
    if(i==0):
        sns.kdeplot(s,c='k',alpha=0.5,
                label='E.g. evolution')

    else:
        sns.kdeplot(s,c='k',alpha=0.5)

plt.plot(X2[0], X1.mean(axis=0), 'b-',lw=3,label='Model pred.')
leg = plt.legend(fontsize=24)
leg.get_frame().set_linewidth(0.0)
plt.yticks([])
plt.xlim([0.2,1.5])
plt.ylabel('')
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
plt.show()

In [ ]:
plt.plot(np.mean(S_MC.mean(axis=-1),axis=0))
plt.errorbar(x = np.arange(8),y = np.mean(S_MC.mean(axis=-1),axis=0), 
            yerr=sp.stats.sem(S_MC[0],axis=1) ,lw=4,c='b')
plt.ylim([0.4,0.6])
plt.yticks(fontsize=48)
plt.xticks(fontsize=48)
plt.show()

### g

In [ ]:
A1 = np.roll(S_MC,-1,axis=-2)[:,:-2,:]-S_MC[:,:-2,:]
A2 = np.roll(S_MC,-2,axis=-2)[:,:-2,:]-np.roll(S_MC,-1,axis=-2)[:,:-2,:]
A1 = np.swapaxes(A1,-2,-1)
A2 = np.swapaxes(A2,-2,-1)
Corr = []
for a1,a2 in zip(A1,A2):
    for i in range(6):
        Corr.append(np.corrcoef(a1[:,i],a2[:,i])[1,0])

a1 = A1[0,np.logical_and(A1[0,:,0]!=0,A2[0,:,0]!=0),0]
a2 = A2[0,np.logical_and(A1[0,:,0]!=0,A2[0,:,0]!=0),0]
plt.scatter(A1[0,np.logical_and(A1[0,:,0]!=0,A2[0,:,0]!=0),0],
            A2[0,np.logical_and(A1[0,:,0]!=0,A2[0,:,0]!=0),0],c='b')
slope, intercept, r_value, p_value, std_err = stats.linregress(a1,a2)
amin = -0.3
amax = 0.26
plt.plot(np.arange(amin,amax,0.01),np.arange(amin,amax,0.01)*slope+intercept,'k',lw=3)

plt.text(x=-0.1,y=0.3,s="Avg. Correlation: {:0.3}".format(np.mean(Corr)),fontsize=24)
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
plt.show()

### h

In [ ]:
def ContLNOU(theta,theta2,mu):
    fmean = sp.interpolate.interp1d(np.arange(-10.0,10.0,0.15),np.arange(-10.0,10.0,0.15)*ml.slope+ml.intercept)
    fstd = sp.interpolate.interp1d(np.arange(-10.0,10.0,0.15),np.arange(-10.0,10.0,0.15)*sl.slope+0.9*sl.intercept)

    S = [Araw[0]]
    for i in range(1,8):
        sMean = fmean(S[-1])+S[-1]
        sStd  = fstd(S[-1])
        lmean = np.log((sMean**2)/np.sqrt(sStd**2+sMean**2))
        lsig = np.sqrt(np.log((sStd/sMean)**2 + 1))
        yi = sp.stats.lognorm.rvs(lsig, loc=-S[-1], scale=np.exp(lmean), size=830)
        nS = S[-1]+(yi)-theta2*(S[-1]-mu)
        if(i>1):
            nS += theta*(S[-2]-S[-1]) 
        S.append(nS)
    return np.array(S)

In [ ]:
S_MC = []
for k in range(1000):
    S_MC.append(ContLNOU(0.2,0.2,0.53))
S_MC = np.array(S_MC)
X1 = [] 
X2 = []
for i,s in enumerate(S_MC[:,-1,:]):
    x = np.histogram(s,bins=15,density=True,range=(np.nanmin(Araw), np.nanmax(Araw)))
    X1.append(x[0]/(sum(x[0])*(x[1][1]-x[1][0])))
    X2.append((x[1][:-1]+x[1][1:])/2)
X1 = np.array(X1)
X2 = np.array(X2)
S1 = [] 
S2 = []
for a in Araw:
    x = np.histogram(a,bins=15,density=True,range=(np.nanmin(Araw), np.nanmax(Araw)))
    S1.append(x[0]/(sum(x[0])*(x[1][1]-x[1][0])))
    S2.append((x[1][:-1]+x[1][1:])/2)
S1 = np.array(S1)
S2 = np.array(S2)
plt.fill_between(X2[0], X1.mean(axis=0)-X1.std(axis=0), X1.mean(axis=0)+X1.std(axis=0),
                 color='b',alpha=0.25)
plt.plot(S2[0], S1.mean(axis=0), 'r-',lw=3,label='Exp. data')

plt.fill_between(S2[0], S1.mean(axis=0)-S1.std(axis=0), S1.mean(axis=0)+S1.std(axis=0),
                 color='r',alpha=0.25)

for i,s in enumerate(S_MC[0,1:,:]):
    if(i==0):
        sns.kdeplot(s,c='k',alpha=0.5,
                label='E.g. evolution')
    else:
        sns.kdeplot(s,c='k',alpha=0.5)
plt.plot(X2[0], X1.mean(axis=0), 'b-',lw=3,label='Model pred.')
leg = plt.legend(fontsize=24)
leg.get_frame().set_linewidth(0.0)
plt.yticks([])
plt.xlim([0.2,1.5])
plt.ylabel('')
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
plt.show()

In [ ]:
plt.errorbar(x = np.arange(8),y = np.mean(S_MC.mean(axis=-1),axis=0), 
             yerr=sp.stats.sem(S_MC[0],axis=1) ,lw=4,c='b')
plt.ylim([0.4,0.6])
plt.yticks(fontsize=48)
plt.xticks(fontsize=48)
plt.show()

### i

In [ ]:
muMC = []
muSTD = []
for s in S_MC:
    Del = []
    for i in range(7):
        Del.append(A[i+1]-A[i])
    DelF = np.vstack(Del).flatten()
    mu, std = stats.norm.fit(DelF)
    muMC.append(mu)
    muSTD.append(std)
col ='k'
A = S_MC[0]
Del = []
for i in range(7):
    Del.append(A[i+1]-A[i])
DelF = np.vstack(Del).flatten()
mu, std = stats.norm.fit(DelF)
x = np.linspace(DelF.min(),DelF.max(),100)
plt.hist(DelF,bins=100,width=0.013,density=True,color='b')
p = stats.norm.pdf(x, mu, std)
plt.plot(x, p, 'k', linewidth=4)
plt.xlim([-0.5,0.5])
plt.ylim([0,7])
plt.text(x=-0.1,y=6.5,s="Avg. Mean: {:0.3}".format(np.mean(muMC)),fontsize=24)
plt.text(x=0.01,y=5.8,s="Avg. STD: {:0.3}".format(np.mean(muSTD)),fontsize=24)
plt.yticks([])
m = stats.lognorm.fit(DelF)
x = np.linspace(DelF.min(),DelF.max())
p = stats.lognorm.pdf(x, m[0], loc=m[1],scale=m[2])
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
plt.show()

### j

In [ ]:
A1 = np.roll(S_MC,-1,axis=-2)[:,:-2,:]-S_MC[:,:-2,:]
A2 = np.roll(S_MC,-2,axis=-2)[:,:-2,:]-np.roll(S_MC,-1,axis=-2)[:,:-2,:]
A1 = np.swapaxes(A1,-2,-1)
A2 = np.swapaxes(A2,-2,-1)
Corr = []
for a1,a2 in zip(A1,A2):
    for i in range(6):
        Corr.append(np.corrcoef(a1[:,i],a2[:,i])[1,0])

a1 = A1[0,np.logical_and(A1[0,:,0]!=0,A2[0,:,0]!=0),0]
a2 = A2[0,np.logical_and(A1[0,:,0]!=0,A2[0,:,0]!=0),0]
plt.scatter(A1[0,np.logical_and(A1[0,:,0]!=0,A2[0,:,0]!=0),0],
            A2[0,np.logical_and(A1[0,:,0]!=0,A2[0,:,0]!=0),0],c='b')
slope, intercept, r_value, p_value, std_err = stats.linregress(a1,a2)
amin = -0.3
amax = 0.26
plt.plot(np.arange(amin,amax,0.01),np.arange(amin,amax,0.01)*slope+intercept,'k',lw=3)

plt.text(x=-0.4,y=0.42,s="Avg. Correlation: {:0.3}".format(np.mean(Corr)),fontsize=24)
plt.xlim([-0.4,0.4])
plt.ylim([-0.4,0.4])
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
plt.show()

## Figure 3

### a

In [ ]:
Dir = BaseDir+'CA1_15Spine/Control/'
D15,A15,(A1raw,_) = ReadFiles(Dir)
A1raw15 = np.vstack(A1raw)
Dir = BaseDir+'CA1_7Spine/Control/'
D7,A7,(A1raw,_) = ReadFiles(Dir)
A1raw7 = np.vstack(A1raw)
Dir = BaseDir+'CA1_3Spine/Control/'
D3,A3,(A1raw,_) = ReadFiles(Dir)
A1raw3 = np.vstack(A1raw)
Dir = BaseDir+'CA1_1Spine/Control/'
D1,A1,(A1raw,_) = ReadFiles(Dir)
A1raw1 = np.vstack(A1raw)
Araw = np.vstack([A1raw1,A1raw3,A1raw7,A1raw15]).T
D    = np.hstack([D1,D3,D7,D15])
D    = D[~np.isnan(Araw.sum(axis=0))]
Araw = Araw[:,~np.isnan(Araw.sum(axis=0))]
D15    = D15[~np.isnan(A1raw15.sum(axis=1))]
A1raw15=A1raw15[~np.isnan(A1raw15.sum(axis=1)),:]

In [ ]:
A=A1raw15.T[:,D15==0]
means = []
sem   = []
var   = []
var_err = []
X = []
for i,a in enumerate(A):
    x = np.histogram(a,bins=15,density=True,range=(np.nanmin(A), np.nanmax(A)))
    means.append(a.mean())
    var.append(a.var())
    var_err.append(stats.bootstrap((a,), np.var, confidence_level=0.95,
                         random_state=1, method='percentile').standard_error)
    sem.append(stats.sem(a))
    X.append(x[0]/(sum(x[0])*(x[1][1]-x[1][0])))
Y  = (x[1][:-1]+x[1][1:])/2
for i,x in enumerate(X):
    if(i<3):
        plt.plot(Y,x,'r',alpha=0.25)
    else:
        plt.plot(Y,x,'b',alpha=0.25)

X1 = X[:3]
X2 = X[3:]
    
plt.plot(Y,np.mean(X1,axis=0),'r',label='pre-stim.',lw=3)
plt.plot(Y,np.mean(X2,axis=0),'b',label='post-stim.',lw=3)
plt.fill_between(Y,np.mean(X1,axis=0)-np.std(X1,axis=0),
                 np.mean(X1,axis=0)+np.std(X1,axis=0),color='r',alpha=0.25)
plt.fill_between(Y,np.mean(X2,axis=0)-np.std(X2,axis=0),
                 np.mean(X2,axis=0)+np.std(X2,axis=0),color='b',alpha=0.25)
leg = plt.legend(fontsize=24)
leg.get_frame().set_linewidth(0.0)
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
ax.set_yticks([])
plt.show()
plt.errorbar(x = np.arange(3,8),y = means[3:], yerr=sem[3:],lw=4,c='b')
plt.errorbar(x = np.arange(3),y = means[:3], yerr=sem[:3],lw=4,c='r')
plt.plot([2,3],[means[2],means[3]],'b',lw=4)
plt.gca().yaxis.tick_right()
ax = plt.gca()
ax.spines["left"].set_visible(False)
ax.spines["top"].set_visible(False)
plt.xticks(fontsize=48)
plt.yticks([0.46,0.50,0.54],fontsize=48)
plt.show()
P1 = [x/sum(x) for x in X1]
P2 = [x/sum(x) for x in X2]
boxprops = dict(linewidth=3, color='b')
whiskerprops = dict(linewidth=3, color='b')
capprops = dict(linewidth=3, color='b')
medianprops = dict( linewidth=3, color='r')
c='r'
exptEnt = [sum(p[p>0]*np.log2(1/p[p>0])) for p in P1]
plt.boxplot([sum(p[p>0]*np.log2(1/p[p>0])) for p in P1],positions=[1],
            bootstrap=1000,boxprops=dict(linewidth=3, color=c),
            capprops=dict(linewidth=3,color=c),
            whiskerprops=dict(linewidth=3,color=c),
            flierprops=dict(linewidth=3,color=c, markeredgecolor=c),
            medianprops=dict(linewidth=3,color='k'),labels=['pre-stim'])
c='b'
plt.boxplot([sum(p[p>0]*np.log2(1/p[p>0])) for p in P2],positions=[2],
            bootstrap=1000,boxprops=dict(linewidth=3, color=c),
            capprops=dict(linewidth=3,color=c),
            whiskerprops=dict(linewidth=3,color=c),
            flierprops=dict(linewidth=3,color=c, markeredgecolor=c),
            medianprops=dict(linewidth=3,color='k'),labels=['post-stim'],showfliers=False)
h = barplot_annotate_brackets(0,1,stats.ttest_ind([sum(p[p>0]*np.log2(1/p[p>0])) for p in P1]
                                        ,[sum(p[p>0]*np.log2(1/p[p>0])) for p in P2]).pvalue,
                               [1,2], [2.75,2.75],fs=64)
ax = plt.gca()
plt.gca().yaxis.tick_right()
plt.xticks(fontsize=64,rotation=-20)
plt.yticks([2.4,2.6,2.8],fontsize=48)
ax.spines["left"].set_visible(False)
ax.spines["top"].set_visible(False)

### b

In [ ]:
A=A1raw15.T[:,np.logical_and(D15!=0,np.abs(D15)<4)] 
means = []
sem   = []
var   = []
var_err = []
X = []
for i,a in enumerate(A):
    x = np.histogram(a,bins=15,density=True,range=(np.nanmin(A), np.nanmax(A)))
    means.append(a.mean())
    var.append(a.var())
    var_err.append(stats.bootstrap((a,), np.var, confidence_level=0.95,
                         random_state=1, method='percentile').standard_error)
    sem.append(stats.sem(a))
    X.append(x[0]/(sum(x[0])*(x[1][1]-x[1][0])))
Y  = (x[1][:-1]+x[1][1:])/2
for i,x in enumerate(X):
    if(i<3):
        plt.plot(Y,x,'r',alpha=0.25)
    else:
        plt.plot(Y,x,'b',alpha=0.25)

X1 = X[:3]
X2 = X[3:]
    
plt.plot(Y,np.mean(X1,axis=0),'r',label='pre-stim.',lw=3)
plt.plot(Y,np.mean(X2,axis=0),'b',label='post-stim.',lw=3)
plt.fill_between(Y,np.mean(X1,axis=0)-np.std(X1,axis=0),
                 np.mean(X1,axis=0)+np.std(X1,axis=0),color='r',alpha=0.25)
plt.fill_between(Y,np.mean(X2,axis=0)-np.std(X2,axis=0),
                 np.mean(X2,axis=0)+np.std(X2,axis=0),color='b',alpha=0.25)
leg = plt.legend(fontsize=24)
leg.get_frame().set_linewidth(0.0)
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
ax.set_yticks([])
plt.show()
plt.errorbar(x = np.arange(3,8),y = means[3:], yerr=sem[3:],lw=4,c='b')
plt.errorbar(x = np.arange(3),y = means[:3], yerr=sem[:3],lw=4,c='r')
plt.plot([2,3],[means[2],means[3]],'b',lw=4)
plt.gca().yaxis.tick_right()
ax = plt.gca()
ax.spines["left"].set_visible(False)
ax.spines["top"].set_visible(False)
plt.xticks(fontsize=48)
plt.yticks([0.46,0.48,0.44],fontsize=48)
plt.show()
P1 = [x/sum(x) for x in X1]
P2 = [x/sum(x) for x in X2]
boxprops = dict(linewidth=3, color='b')
whiskerprops = dict(linewidth=3, color='b')
capprops = dict(linewidth=3, color='b')
medianprops = dict( linewidth=3, color='r')
c='r'
plt.boxplot([sum(p[p>0]*np.log2(1/p[p>0])) for p in P1],positions=[1],
            bootstrap=1000,boxprops=dict(linewidth=3, color=c),
            capprops=dict(linewidth=3,color=c),
            whiskerprops=dict(linewidth=3,color=c),
            flierprops=dict(linewidth=3,color=c, markeredgecolor=c),
            medianprops=dict(linewidth=3,color='k'),labels=['pre-stim'])
c='b'
plt.boxplot([sum(p[p>0]*np.log2(1/p[p>0])) for p in P2],positions=[2],
            bootstrap=1000,boxprops=dict(linewidth=3, color=c),
            capprops=dict(linewidth=3,color=c),
            whiskerprops=dict(linewidth=3,color=c),
            flierprops=dict(linewidth=3,color=c, markeredgecolor=c),
            medianprops=dict(linewidth=3,color='k'),labels=['post-stim'],showfliers=False)
h = barplot_annotate_brackets(0,1,stats.ttest_ind([sum(p[p>0]*np.log2(1/p[p>0])) for p in P1]
                                                  ,[sum(p[p>0]*np.log2(1/p[p>0])) for p in P2]).pvalue,
                               [1,2], [3.15,3.15],fs=64)
ax = plt.gca()
plt.gca().yaxis.tick_right()
plt.xticks(fontsize=64,rotation=-20)
plt.yticks(fontsize=48)
ax.spines["left"].set_visible(False)
ax.spines["top"].set_visible(False)

### c

In [ ]:
col ='k'
A = A1raw15.T[:,D15==0]
Del = []
for i in range(7):
    Del.append(A[i+1]-A[i])
Del = np.array(Del)
fig = plt.figure(figsize=(24,8))
ax = fig.add_subplot(111, projection='3d')
nbins = 50
red = np.array([1,0,0])
blue = np.array([0,0,1])
muvec = []
sem = []
for i,(z,d) in enumerate(zip([120,100,80,60, 40, 20, 0],Del)):
    hist, bins = np.histogram(d, bins=nbins,density=True)
    xs = (bins[:-1] + bins[1:])/2
    if(i<2):
        col=red
    elif(i==2):
        col='teal'
    else:
        col=blue
    ax.bar(xs, hist, zs=z, zdir='x', 
           color=col,alpha=0.6,width=(xs[-1]-xs[0])/50)
    mu, std = stats.norm.fit(d)
    x = np.linspace(d.min(),d.max())
    p = stats.norm.pdf(x, mu, std)
    ax.plot(x,p,zs=z, zdir='x', linewidth=2,c='k')
    x = [z]*5
    y = [mu]*5
    z2 = np.linspace(0,p.max(),5)
    ax.plot(x,y,z2,'k-',linewidth=2)
    ax.yaxis.labelpad = 10
    muvec.append(mu)
    sem.append(stats.sem(d))
ax.view_init(elev=30., azim=-50)

ax.set_xticks([])
ax.set_zticks([])
ax.set_yticks([-0.5,0.,0.5])

ax.get_proj = lambda: np.dot(Axes3D.get_proj(ax), np.diag([1.8, 1, 1, 1.4]))
plt.show()
fig,ax = plt.subplots(figsize=(6,2))
plt.errorbar(x = np.arange(3,7),y = muvec[3:], yerr=sem[3:],lw=4,c='b')
plt.errorbar(x = np.arange(2),y = muvec[:2], yerr=sem[:2],lw=4,c='r')
plt.errorbar(x = [2],y = muvec[2], yerr=sem[2],lw=4,c='teal')
plt.plot([1,2,3],[muvec[1],muvec[2],muvec[3]],'teal',lw=4)
plt.plot([0,1],[0.09,0.09],'k-')
plt.plot([3,6],[0.09,0.09],'k-')
barplot_annotate_brackets(0, 1, 
                          stats.f_oneway(Del[0],Del[1],Del[3],Del[4],Del[5],Del[6]).pvalue, 
                          [0.5,4.5],[0.1]*2,fs=36)
barplot_annotate_brackets(0, 1, stats.f_oneway(Del[0],Del[1],Del[2],Del[3],Del[4],Del[5],Del[6]).pvalue, 
                          [2,2],[0.03]*2,fs=36)
ax.set_ylim(ax.get_ylim()[0],ax.get_ylim()[1]+0.02)
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)

### d

In [ ]:
col ='k'
A = A1raw15.T[:,np.logical_and(D15!=0,abs(D15)<4)]
Del = []
for i in range(7):
    Del.append(A[i+1]-A[i])
Del = np.array(Del)
fig = plt.figure(figsize=(24,8))
ax = fig.add_subplot(111, projection='3d')
nbins = 25
red = np.array([1,0,0])
blue = np.array([0,0,1])
muvec = []
sem = []
for i,(z,d) in enumerate(zip([120,100,80,60, 40, 20, 0],Del)):
    hist, bins = np.histogram(d, bins=nbins,density=True)
    xs = (bins[:-1] + bins[1:])/2
    if(i<2):
        col=red
    elif(i==2):
        col='teal'
    else:
        col=blue
    ax.bar(xs, hist, zs=z, zdir='x', 
           color=col,alpha=0.6,width=(xs[-1]-xs[0])/nbins)
    mu, std = stats.norm.fit(d)
    x = np.linspace(d.min(),d.max())
    p = stats.norm.pdf(x, mu, std)
    ax.plot(x,p,zs=z, zdir='x', linewidth=2,c='k')
    x = [z]*5
    y = [mu]*5
    z2 = np.linspace(0,p.max(),5)
    ax.plot(x,y,z2,'k-',linewidth=2)
    ax.yaxis.labelpad = 10
    muvec.append(mu)
    sem.append(stats.sem(d))
ax.view_init(elev=30., azim=-50)

ax.set_xticks([])
ax.set_zticks([])
ax.set_yticks([-0.5,0.,0.5])

ax.get_proj = lambda: np.dot(Axes3D.get_proj(ax), np.diag([1.8, 1, 1, 1.4]))

plt.show()
fig,ax = plt.subplots(figsize=(6,2))
plt.errorbar(x = np.arange(3,7),y = muvec[3:], yerr=sem[3:],lw=4,c='b')
plt.errorbar(x = np.arange(2),y = muvec[:2], yerr=sem[:2],lw=4,c='r')
plt.errorbar(x = [2],y = muvec[2], yerr=sem[2],lw=4,c='teal')
plt.plot([1,2,3],[muvec[1],muvec[2],muvec[3]],'teal',lw=4)
plt.plot([0,6],[0.05,0.05],'k-')
barplot_annotate_brackets(0, 1, 
                          stats.f_oneway(Del[0],Del[1],Del[2],Del[3],Del[4],Del[5],Del[6]).pvalue, 
                          [3,3],[0.05]*2,fs=48)


ax.set_ylim(ax.get_ylim()[0],ax.get_ylim()[1]+0.01)
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)

### e

In [ ]:
Del_tot = []
for Cutoff in np.arange(0.2,0.8,0.15):
    col ='k'
    A = A1raw15[D15==0,:].T
    Del1 = []
    for i in range(7):
        mask = np.logical_and(A[i]>Cutoff,A[i]<Cutoff+0.15)
        Del1.append(A[i+1,mask]-A[i,mask])
    Del_tot.append(Del1)
Del_tot = np.stack(Del_tot)
Del_wo = np.delete(Del_tot,2, axis=-1)
fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(111, projection='3d')
nbins = 25
red = np.array([1,0,0])
blue = np.array([0,0,1])
P2 = []
M2 = []
for i,(z,d) in enumerate(zip([0.2,0.35,0.5,0.65],Del_wo)):
    d = np.hstack(d)
    M2.append(d.mean())
    hist, bins = np.histogram(d, bins=25,density=True)
    xs = (bins[:-1] + bins[1:])/2
    m = stats.lognorm.fit(d)
    ax.bar(xs, hist, zs=z, zdir='x', color=red,alpha=0.6,width=(xs[-1]-xs[0])/nbins)
    x = np.linspace(-0.5,0.6)
    p1 = stats.lognorm.pdf(x, m[0], loc=m[1],scale=m[2])
    P2.append(p1)
    ax.plot(x,p1,zs=z, zdir='x', linewidth=2,c='k')
ax.view_init(elev=30., azim=-30)
ax.set_ylim(-0.6,0.6)

ax.set_zticks([])
ax.yaxis.labelpad = 10
ax.xaxis.labelpad = 10
plt.gca().invert_xaxis()
plt.yticks(fontsize=40)
plt.xticks(fontsize=40)
plt.show()

fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(111, projection='3d')
nbins = 10
red = np.array([1,0,0])
blue = np.array([0,0,1])
P = []
M = []
for i,(z,d) in enumerate(zip([0.2,0.35,0.5,0.65],Del_tot[:,2])):
    d = np.hstack(d)
    M.append(d.mean())
    hist, bins = np.histogram(d, bins=nbins,density=True)
    xs = (bins[:-1] + bins[1:])/2
    m = stats.lognorm.fit(d)
    ax.bar(xs, hist, zs=z, zdir='x', color='teal',alpha=0.6,width=(xs[-1]-xs[0])/nbins)
    x = np.linspace(-0.5,0.6)
    p1 = stats.lognorm.pdf(x, m[0], loc=m[1],scale=m[2])
    P.append(p1)
    ax.plot(x,p1,zs=z, zdir='x', linewidth=2,c='k')
ax.view_init(elev=30., azim=-30)
ax.set_ylim(-0.6,0.6)

ax.set_zticks([])
ax.yaxis.labelpad = 10
ax.xaxis.labelpad = 10
plt.gca().invert_xaxis()
plt.yticks(fontsize=40)
plt.xticks(fontsize=40)
plt.show()

### f

In [ ]:
fig,axs = plt.subplots(2,2,figsize=(12.8,9.6))
ax = axs.ravel()
for i,(p1,p2,m1,m2,a) in enumerate(zip(P,P2,M,M2,ax)):
    if(i==2):
        a.plot(x,p1,lw=5,c='teal',label='stimulated \n snapshot')
        a.plot(x,p2,lw=5,c='r')
    elif(i==3):
        a.plot(x,p1,lw=5,c='teal')
        a.plot(x,p2,lw=5,c='r',label='non-stimulated \n snapshots')
    else:
        a.plot(x,p1,lw=5,c='teal',label='stimulated \n snapshot')
        a.plot(x,p2,lw=5,c='r',label='non-stimulated \n snapshots')
    a.axvline(m1,c='teal',ls='--',lw=4)
    a.axvline(m2,c='r',ls='--',lw=4)
    a.set_ylim((-0.34867040846353875, 7.322078577734313))
    a.spines["right"].set_visible(False)
    a.spines["top"].set_visible(False)
    a.tick_params(axis='x', labelsize=40)
    a.set_yticks([])
leg = ax[-2].legend(fontsize=36)
leg.get_frame().set_linewidth(0.0)
leg = ax[-1].legend(fontsize=36)
leg.get_frame().set_linewidth(0.0)

### g

In [ ]:
Del_tot = []
A = A1raw15.T[:,np.logical_and(D15!=0,abs(D15)<4)]
for c1,c2 in zip([0,0.35,0.5,0.65],[0.35,0.5,0.65,100]):
    col ='k'
    Del1 = []
    for i in range(7):
        mask = np.logical_and(A[i]>c1,A[i]<c2)
        Del1.append(A[i+1,mask]-A[i,mask])
    Del_tot.append(Del1)
Del_tot = np.stack(Del_tot)
Del_wo = np.delete(Del_tot,2, axis=-1)

fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(111, projection='3d')
nbins = 25
red = np.array([1,0,0])
blue = np.array([0,0,1])
P2 = []
M2 = []
for i,(z,d) in enumerate(zip([0.2,0.35,0.5,0.65],Del_wo)):
    d = np.hstack(d)
    M2.append(d.mean())
    hist, bins = np.histogram(d, bins=25,density=True)
    xs = (bins[:-1] + bins[1:])/2
    m = stats.lognorm.fit(d)
    ax.bar(xs, hist, zs=z, zdir='x', color=red,alpha=0.6,width=(xs[-1]-xs[0])/nbins)
    x = np.linspace(-0.5,0.6)
    p1 = stats.lognorm.pdf(x, m[0], loc=m[1],scale=m[2])
    P2.append(p1)
    ax.plot(x,p1,zs=z, zdir='x', linewidth=2,c='k')
ax.view_init(elev=30., azim=-30)
ax.set_ylim(-0.6,0.6)
ax.set_zticks([])
ax.yaxis.labelpad = 10
ax.xaxis.labelpad = 10
plt.yticks(fontsize=40)
plt.xticks(fontsize=40)
plt.gca().invert_xaxis()

fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(111, projection='3d')
nbins = 10
red = np.array([1,0,0])
blue = np.array([0,0,1])
P = []
M = []
for i,(z,d) in enumerate(zip([0.2,0.35,0.5,0.65],Del_tot[:,2])):
    d = np.hstack(d)
    M.append(d.mean())
    hist, bins = np.histogram(d, bins=nbins,density=True)
    xs = (bins[:-1] + bins[1:])/2
    m = stats.lognorm.fit(d)
    ax.bar(xs, hist, zs=z, zdir='x', color='teal',alpha=0.6,width=(xs[-1]-xs[0])/nbins)
    x = np.linspace(-0.5,0.6)
    p1 = stats.lognorm.pdf(x, m[0], loc=m[1],scale=m[2])
    P.append(p1)
    ax.plot(x,p1,zs=z, zdir='x', linewidth=2,c='k')
ax.view_init(elev=30., azim=-30)
ax.set_ylim(-0.6,0.6)
ax.set_zticks([])
ax.yaxis.labelpad = 10
ax.xaxis.labelpad = 10
plt.yticks(fontsize=40)
plt.xticks(fontsize=40)
plt.gca().invert_xaxis()

### h

In [ ]:
fig,axs = plt.subplots(2,2,figsize=(12.8,9.6))
ax = axs.ravel()
for i,(p1,p2,m1,m2,a) in enumerate(zip(P,P2,M,M2,ax)):
    if(i==2):
        a.plot(x,p1,lw=5,c='teal',label='stimulated \n snapshot')
        a.plot(x,p2,lw=5,c='r')
    elif(i==3):
        a.plot(x,p1,lw=5,c='teal')
        a.plot(x,p2,lw=5,c='r',label='non-stimulated \n snapshots')
    else:
        a.plot(x,p1,lw=5,c='teal',label='stimulated \n snapshot')
        a.plot(x,p2,lw=5,c='r',label='non-stimulated \n snapshots')
    a.axvline(m1,c='teal',ls='--',lw=4)
    a.axvline(m2,c='r',ls='--',lw=4)
    a.set_ylim((-0.41984416128529756, 8.816727386991248))
    a.spines["right"].set_visible(False)
    a.spines["top"].set_visible(False)
    a.tick_params(axis='x', labelsize=40)
    a.set_yticks([])

## Figure 4

### a

In [ ]:
lowermean = []
lowerstd  = []
semMean = []
semSTD  = []
for Cutoff in np.arange(0.2,0.66,0.15):
    col ='k'
    A = A1raw15[D15==0].T
    Del1 = []
    for i in [0,1,3,4,5,6]:
        mask = np.logical_and(A[i]>Cutoff,A[i]<Cutoff+0.15)
        Del1.append(A[i+1,mask]-A[i,mask])
    DelF = np.hstack(Del1).flatten()
    mu, std = stats.norm.fit(DelF)
    semMean.append(stats.bootstrap((DelF,), np.mean, confidence_level=0.95,
                         random_state=1, method='percentile').standard_error)
    semSTD.append(stats.bootstrap((DelF,), np.std, confidence_level=0.95,
                     random_state=1, method='percentile').standard_error)
    lowermean.append(mu)
    lowerstd.append(std)
ml = stats.linregress(np.arange(0.2,0.66,0.15),lowermean)
sl = stats.linregress(np.arange(0.2,0.66,0.15),lowerstd)
plt.errorbar(np.arange(0.2,0.66,0.15),lowermean,semMean,capsize=5,ls='None',ms=8,fmt='.',color='r')
plt.plot(np.arange(0.2,0.66,0.15),np.arange(0.2,0.66,0.15)*ml.slope+ml.intercept,ls='--',c=blue+[0.5,0.5,0],lw=3,label='Best fit')
plt.errorbar(np.arange(0.2,0.66,0.15),lowerstd,semSTD,capsize=5,ls='None',ms=8,fmt='.',c='r')
plt.plot(np.arange(0.2,0.66,0.15),np.arange(0.2,0.66,0.15)*sl.slope+sl.intercept,ls='--',c=blue*0.5,lw=3)
print("Slope of mean: {:.3} and intercept: {:.3}".format(ml.slope,ml.intercept))
print("Slope of std: {:.3} and intercept: {:.3}".format(sl.slope,sl.intercept))
plt.plot(np.arange(0.2,0.66,0.15),
         np.arange(0.2,0.66,0.15)*-0.2026966726161646+0.09590843817644161/1.2,lw=3,c='k',alpha=0.5,label='Best fit \n from sham')
plt.plot(np.arange(0.2,0.66,0.15),np.arange(0.2,0.66,0.15)*0.15289944615303516+0.029179477272253437,lw=3,c='k',alpha=0.5)

lowermean2 = lowermean
lowerstd2  = lowerstd
leg = plt.legend(loc=5,fontsize=24)
leg.get_frame().set_linewidth(0.0)
plt.text(x=0.23,y=0.08,s="Fit of std. of change",rotation=13,fontsize=28)
plt.text(x=0.24,y=-0.05,s="Fit of mean changes",rotation=-18,fontsize=28)
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
plt.xticks(fontsize=24)
plt.yticks(fontsize=24)
plt.show()

### b

In [ ]:
lowermean = []
lowerstd  = []
semMean = []
semSTD  = []
for Cutoff in np.arange(0.2,0.66,0.15):
    col ='k'
    A = A1raw15.T[:,np.logical_and(D15!=0,abs(D15)<4)]
    Del1 = []
    for i in [0,1,3,4,5,6]:
        mask = np.logical_and(A[i]>Cutoff,A[i]<Cutoff+0.15)
        Del1.append(A[i+1,mask]-A[i,mask])
    DelF = np.hstack(Del1).flatten()
    mu, std = stats.norm.fit(DelF)
    semMean.append(stats.bootstrap((DelF,), np.mean, confidence_level=0.95,
                         random_state=1, method='percentile').standard_error)
    semSTD.append(stats.bootstrap((DelF,), np.std, confidence_level=0.95,
                     random_state=1, method='percentile').standard_error)
    lowermean.append(mu)
    lowerstd.append(std)
ml = stats.linregress(np.arange(0.2,0.66,0.15),lowermean)
sl = stats.linregress(np.arange(0.2,0.66,0.15),lowerstd)
plt.errorbar(np.arange(0.2,0.66,0.15),lowermean,semMean,capsize=5,ls='None',ms=8,fmt='.',color='r')
plt.plot(np.arange(0.2,0.66,0.15),np.arange(0.2,0.66,0.15)*ml.slope+ml.intercept,ls='--',c=blue+[0.5,0.5,0],lw=3,label='Best fit')
plt.errorbar(np.arange(0.2,0.66,0.15),lowerstd,semSTD,capsize=5,ls='None',ms=8,fmt='.',c='r')
plt.plot(np.arange(0.2,0.66,0.15),np.arange(0.2,0.66,0.15)*sl.slope+sl.intercept,ls='--',c=blue*0.5,lw=3)

print("Slope of mean: {:.3} and intercept: {:.3}".format(ml.slope,ml.intercept))
print("Slope of std: {:.3} and intercept: {:.3}".format(sl.slope,sl.intercept))
plt.plot(np.arange(0.2,0.66,0.15),
         np.arange(0.2,0.66,0.15)*-0.2026966726161646+0.09590843817644161/1.2,lw=3,c='k',alpha=0.5,label='Best fit \n from sham')
plt.plot(np.arange(0.2,0.66,0.15),np.arange(0.2,0.66,0.15)*0.15289944615303516+0.029179477272253437,lw=3,c='k',alpha=0.5)

lowermean2 = lowermean
lowerstd2  = lowerstd
plt.text(x=0.23,y=0.08,s="Fit of std. of changes",rotation=12,fontsize=28)
plt.text(x=0.24,y=-0.065,s="Fit of mean changes",rotation=-18,fontsize=28)
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
plt.xticks(fontsize=24)
plt.yticks(fontsize=24)
plt.show()

###  c

In [ ]:
fig,ax = plt.subplots(2,1)
plt.subplots_adjust(wspace=0, hspace=0)
lowermean = []
lowerstd  = []
semMean = []
semSTD  = []
for Cutoff in np.arange(0.2,0.66,0.15):
    col ='k'
    A = A1raw15.T[:,np.logical_and(D15!=0,abs(D15)<4)]
    Del1 = []
    for i in [2]:
        mask = np.logical_and(A[i]>Cutoff,A[i]<Cutoff+0.15)
        Del1.append(A[i+1,mask]-A[i,mask])
    DelF = np.hstack(Del1).flatten()
    mu, std = stats.norm.fit(DelF)
    semMean.append(stats.bootstrap((DelF,), np.mean, confidence_level=0.95,
                         random_state=1, method='percentile').standard_error)
    semSTD.append(stats.bootstrap((DelF,), np.std, confidence_level=0.95,
                     random_state=1, method='percentile').standard_error)
    lowermean.append(mu)
    lowerstd.append(std)
ml = stats.linregress(np.arange(0.2,0.66,0.15),lowermean)
sl = stats.linregress(np.arange(0.2,0.66,0.15),lowerstd)
ax[0].set_xticks([])
plt.errorbar(np.arange(0.2,0.66,0.15),lowermean,semMean,capsize=5,ls='None',ms=8,fmt='.',color='teal')
plt.plot(np.arange(0.2,0.66,0.15),np.arange(0.2,0.66,0.15)*ml.slope+ml.intercept,ls='--',c=blue+[0.5,0.5,0],lw=3,label='Best fit')
ax[0].errorbar(np.arange(0.2,0.66,0.15),lowerstd,semSTD,capsize=5,ls='None',ms=8,fmt='.',c='teal')
ax[0].plot(np.arange(0.2,0.66,0.15),np.arange(0.2,0.66,0.15)*sl.slope+sl.intercept,ls='--',c=blue*0.5,lw=3)
print("Slope of mean: {:.3} and intercept: {:.3}".format(ml.slope,ml.intercept))
print("Slope of std: {:.3} and intercept: {:.3}".format(sl.slope,sl.intercept))
plt.plot(np.arange(0.2,0.66,0.15),
         np.arange(0.2,0.66,0.15)*-0.2026966726161646+0.09590843817644161/1.2,lw=3,c='k',alpha=0.5,label='Best fit \n from sham')
ax[0].plot(np.arange(0.2,0.66,0.15),np.arange(0.2,0.66,0.15)*0.15289944615303516+0.029179477272253437,lw=3,c='k',alpha=0.5,label='Best fit \n from sham')

plt.xticks(fontsize=24)
plt.yticks(fontsize=24)
ax = plt.gca()
plt.show()

### d

In [ ]:
fig,ax = plt.subplots(2,1)
plt.subplots_adjust(wspace=0, hspace=0)
lowermean = []
lowerstd  = []
semMean = []
semSTD  = []
for Cutoff in np.arange(0.2,0.66,0.15):
    col ='k'
    A = A1raw15[D15==0].T
    Del1 = []
    for i in [2]:
        mask = np.logical_and(A[i]>Cutoff,A[i]<Cutoff+0.15)
        Del1.append(A[i+1,mask]-A[i,mask])
    DelF = np.hstack(Del1).flatten()
    mu, std = stats.norm.fit(DelF)
    semMean.append(stats.bootstrap((DelF,), np.mean, confidence_level=0.95,
                         random_state=1, method='percentile').standard_error)
    semSTD.append(stats.bootstrap((DelF,), np.std, confidence_level=0.95,
                     random_state=1, method='percentile').standard_error)
    lowermean.append(mu)
    lowerstd.append(std)
ml = stats.linregress(np.arange(0.2,0.66,0.15),lowermean)
sl = stats.linregress(np.arange(0.2,0.66,0.15),lowerstd)
ax[0].set_xticks([])
plt.errorbar(np.arange(0.2,0.66,0.15),lowermean,semMean,capsize=5,ls='None',ms=8,fmt='.',color='teal')
plt.plot(np.arange(0.2,0.66,0.15),np.arange(0.2,0.66,0.15)*ml.slope+ml.intercept,ls='--',c=blue+[0.5,0.5,0],lw=3,label='Best fit')
ax[0].errorbar(np.arange(0.2,0.66,0.15),lowerstd,semSTD,capsize=5,ls='None',ms=8,fmt='.',c='teal')
ax[0].plot(np.arange(0.2,0.66,0.15),np.arange(0.2,0.66,0.15)*sl.slope+sl.intercept,ls='--',c=blue*0.5,lw=3)
print("Slope of mean: {:.3} and intercept: {:.3}".format(ml.slope,ml.intercept))
print("Slope of std: {:.3} and intercept: {:.3}".format(sl.slope,sl.intercept))
plt.plot(np.arange(0.2,0.66,0.15),
         np.arange(0.2,0.66,0.15)*-0.2026966726161646+0.09590843817644161/1.2,lw=3,c='k',alpha=0.5,label='Best fit \n from sham')
ax[0].plot(np.arange(0.2,0.66,0.15),np.arange(0.2,0.66,0.15)*0.15289944615303516+0.029179477272253437,lw=3,c='k',alpha=0.5,label='Best fit \n from sham')
plt.xticks(fontsize=24)
plt.yticks(fontsize=24)
plt.show()

### e

In [ ]:
BaseDir = '/Users/maximilianeggl/Dropbox/PostDoc/HeteroPlast/heterosynaptic_project/MaxProj/'
Dir = BaseDir+'CA1_15Spine/Sham/'
D15,A1,(A1raw,_) = ReadFiles(Dir)
A1raw15 = np.vstack(A1raw)
Dir = BaseDir+'CA1_7Spine/Sham/'
D7,A1,(A1raw,_) = ReadFiles(Dir)
A1raw7 = np.vstack(A1raw)
Dir = BaseDir+'CA1_3Spine/Sham/'
D3,A1,(A1raw,_) = ReadFiles(Dir)
A1raw3 = np.vstack(A1raw)
Dir = BaseDir+'CA1_1Spine/Sham/'
D1,A1,(A1raw,_) = ReadFiles(Dir)
A1raw1 = np.vstack(A1raw)
Araw = np.vstack([A1raw1,A1raw3,A1raw7,A1raw15]).T
Araw = Araw[:,~np.isnan(Araw.sum(axis=0))]

lowermean = []
lowerstd  = []
uppermean = []
upperstd  = []
semMean = []
semSTD  = []
for Cutoff in np.arange(0.2,1.0,0.15):
    col ='k'
    A = Araw
    Del1 = []
    for i in range(7):
        mask = np.logical_and(A[i]>Cutoff,A[i]<Cutoff+0.15)
        Del1.append(A[i+1,mask]-A[i,mask])
    DelF = np.hstack(Del1).flatten()
    DelF = np.hstack(Del1).flatten()
    mu, std = stats.norm.fit(DelF)
    semMean.append(stats.bootstrap((DelF,), np.mean, confidence_level=0.95,
                         random_state=1, method='percentile').standard_error)
    semSTD.append(stats.bootstrap((DelF,), np.std, confidence_level=0.95,
                     random_state=1, method='percentile').standard_error)
    lowermean.append(mu)
    lowerstd.append(std)
ml = stats.linregress(np.arange(0.2,1.0,0.15),lowermean)
sl = stats.linregress(np.arange(0.2,1.0,0.15),lowerstd)

In [ ]:
def ContLNStim(theta,theta2,mu,mu_add=0):
    fmean = sp.interpolate.interp1d(np.arange(-5.0,5.0,0.15),np.arange(-5.0,5.0,0.15)*ml.slope+ml.intercept)
    fstd = sp.interpolate.interp1d(np.arange(-5.0,5.0,0.15),np.arange(-5.0,5.0,0.15)*sl.slope+sl.intercept)
    fmean2 = sp.interpolate.interp1d(np.arange(-5.0,5.0,0.15),
                                    np.arange(-5.0,5.0,0.15)*-0.33988898633439907+0.17344471005740245)
    fstd2 = sp.interpolate.interp1d(np.arange(-5.0,5.0,0.15),
                                    np.arange(-5.0,5.0,0.15)*0.12238578174002662+0.05526252834977198)


    S = [A1raw15.T[0]]
    for i in range(1,8):
        if(i==3):
            sMean = fmean2(S[-1])+S[-1]
            sStd  = fstd2(S[-1])
            mu+=mu_add
        else:
            sMean = fmean(S[-1])+S[-1]
            sStd  = fstd(S[-1])
        lmean = np.log((sMean**2)/np.sqrt(sStd**2+sMean**2))
        lsig = np.sqrt(np.log((sStd/sMean)**2 + 1))
        yi = sp.stats.lognorm.rvs(lsig, loc=-S[-1], scale=np.exp(lmean), size=len(S[0]))
        nS = S[-1]+(yi)-theta2*(S[-1]-mu)
        if(i>1 and i!=3):
            nS += theta*(S[-2]-S[-1]) 
        S.append(nS)
    return np.array(S)

def ModelPlot(theta,mu,mu_add=0):
    S = ContLNStim(theta,mu,mu_add)
    pltDist(S)
    pltDyn(S)
    return S

In [ ]:
S_MC = []
for k in range(1000):
    S_MC.append(ContLNStim(0.2,0.11,0.6))
S_MC = np.array(S_MC)
XX1 = []
for i in range(S_MC.shape[-2]):
    X1 = [] 
    for i,s in enumerate(S_MC[:,i,:]):
        x = np.histogram(s,bins=15,density=True,range=(np.nanmin(Araw), np.nanmax(Araw)))
        X1.append(x[0]/(sum(x[0])*(x[1][1]-x[1][0])))
    X2 = (x[1][:-1]+x[1][1:])/2
    XX1.append(X1)
for i,s in enumerate(S_MC[0,:,:]):
    x = np.histogram(s,bins=15,density=True,range=(np.nanmin(s), np.nanmax(s)))
    if(i<3):
        sns.kdeplot(s,c='r',alpha=0.5)
    else:
        sns.kdeplot(s,c='b',alpha=0.5)
plt.plot(X2,np.vstack(XX1[0]+XX1[1]+XX1[2]).mean(axis=0),'r',label='Pre-stim.',lw=3)
plt.fill_between(X2, np.vstack(XX1[0]+XX1[1]+XX1[2]).mean(axis=0)-np.vstack(XX1[0]+XX1[1]+XX1[2]).std(axis=0),
                 np.vstack(XX1[0]+XX1[1]+XX1[2]).mean(axis=0)+np.vstack(XX1[0]+XX1[1]+XX1[2]).std(axis=0),
                 color='r',alpha=0.25)
plt.plot(X2,np.vstack(XX1[3]+XX1[4]+XX1[6]+XX1[4]+XX1[7]).mean(axis=0),'b',label='Post-stim.',lw=3)
plt.fill_between(X2, np.vstack(XX1[3]+XX1[4]+XX1[6]+XX1[4]+XX1[7]).mean(axis=0)-np.vstack(XX1[3]+XX1[4]+XX1[6]+XX1[4]+XX1[7]).std(axis=0),
                 np.vstack(XX1[3]+XX1[4]+XX1[6]+XX1[4]+XX1[7]).mean(axis=0)+np.vstack(XX1[3]+XX1[4]+XX1[6]+XX1[4]+XX1[7]).std(axis=0),
                 color='b',alpha=0.25)
plt.xlim([0.2,1.5])
leg = plt.legend(fontsize=24)
leg.get_frame().set_linewidth(0.0)
plt.yticks([])
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
plt.ylabel('')
plt.show()

t = np.random.choice(1000,1000)
X1 = np.array(XX1)[:3,t,:]
X2 = np.array(XX1)[3,t,:]
X3 = np.array(XX1)[-1,t,:]
T = []
for i in range(X1.shape[-2]):
    P1 = [x/sum(x) for x in X1[:,i,:]]
    T.append(np.array([sum(p[p>0]*np.log2(1/p[p>0])) for p in P1]).mean(axis=0))
    
T1_s = []
for x in X2:
    P1 = x/sum(x)
    T1_s.append([sum(p[p>0]*np.log2(1/p[p>0])) for p in [P1]])

T1_e = []
for x in X3:
    P1 = x/sum(x)
    T1_e.append([sum(p[p>0]*np.log2(1/p[p>0])) for p in [P1]])
        

In [ ]:
means = np.mean(S_MC.mean(axis=-1),axis=0)
sem   = sp.stats.sem(S_MC[0],axis=1)
plt.errorbar(x = np.arange(3,8),y = means[3:], yerr=sem[3:],lw=4,c='b')
plt.errorbar(x = np.arange(3),y = means[:3], yerr=sem[:3],lw=4,c='r')
plt.plot([2,3],[means[2],means[3]],'b',lw=4)
plt.ylim([0.46,0.54])
plt.yticks(fontsize=48)
plt.xticks(fontsize=48)
plt.show()

### f

In [ ]:
def ContLNStimMu(theta,theta2,mu,mu_add=0):
    fmean = sp.interpolate.interp1d(np.arange(-5.0,5.0,0.15),np.arange(-5.0,5.0,0.15)*ml.slope+ml.intercept)
    fstd = sp.interpolate.interp1d(np.arange(-5.0,5.0,0.15),np.arange(-5.0,5.0,0.15)*sl.slope+sl.intercept)
    fmean2 = sp.interpolate.interp1d(np.arange(-5.0,5.0,0.15),
                                    np.arange(-5.0,5.0,0.15)*-0.33988898633439907+0.17344471005740245)
    fstd2 = sp.interpolate.interp1d(np.arange(-5.0,5.0,0.15),
                                    np.arange(-5.0,5.0,0.15)*0.12238578174002662+0.05526252834977198)


    S = [A1raw15.T[0]]
    for i in range(1,8):
        if(i==3):
            mu+=mu_add
            sMean = fmean(S[-1])+S[-1]
            sStd  = fstd(S[-1])
        else:
            sMean = fmean(S[-1])+S[-1]
            sStd  = fstd(S[-1])
        lmean = np.log((sMean**2)/np.sqrt(sStd**2+sMean**2))
        lsig = np.sqrt(np.log((sStd/sMean)**2 + 1))
        yi = sp.stats.lognorm.rvs(lsig, loc=-S[-1], scale=np.exp(lmean), size=len(S[0]))
        nS = S[-1]+(yi)-theta2*(S[-1]-mu)
        if(i>1 and i!=3):
            nS += theta*(S[-2]-S[-1]) 
        S.append(nS)
    return np.array(S)

S_MC = []
for k in range(1000):
    S_MC.append(ContLNStimMu(0.2,0.11,0.6,0.14589222947969693))
S_MC = np.array(S_MC)
XX1 = []
for i in range(S_MC.shape[-2]):
    X1 = [] 
    for i,s in enumerate(S_MC[:,i,:]):
        x = np.histogram(s,bins=15,density=True,range=(np.nanmin(Araw), np.nanmax(Araw)))
        X1.append(x[0]/(sum(x[0])*(x[1][1]-x[1][0])))
    X2 = (x[1][:-1]+x[1][1:])/2
    XX1.append(X1)
for i,s in enumerate(S_MC[0,:,:]):
    x = np.histogram(s,bins=15,density=True,range=(np.nanmin(s), np.nanmax(s)))
    if(i<3):
        sns.kdeplot(s,c='r',alpha=0.5)
    else:
        sns.kdeplot(s,c='b',alpha=0.5)
plt.plot(X2,np.vstack(XX1[0]+XX1[1]+XX1[2]).mean(axis=0),'r',label='Pre-stim.',lw=3)
plt.fill_between(X2, np.vstack(XX1[0]+XX1[1]+XX1[2]).mean(axis=0)-np.vstack(XX1[0]+XX1[1]+XX1[2]).std(axis=0),
                 np.vstack(XX1[0]+XX1[1]+XX1[2]).mean(axis=0)+np.vstack(XX1[0]+XX1[1]+XX1[2]).std(axis=0),
                 color='r',alpha=0.25)
plt.plot(X2,np.vstack(XX1[3]+XX1[4]+XX1[6]+XX1[4]+XX1[7]).mean(axis=0),'b',label='Post-stim.',lw=3)
plt.fill_between(X2, np.vstack(XX1[3]+XX1[4]+XX1[6]+XX1[4]+XX1[7]).mean(axis=0)-np.vstack(XX1[3]+XX1[4]+XX1[6]+XX1[4]+XX1[7]).std(axis=0),
                 np.vstack(XX1[3]+XX1[4]+XX1[6]+XX1[4]+XX1[7]).mean(axis=0)+np.vstack(XX1[3]+XX1[4]+XX1[6]+XX1[4]+XX1[7]).std(axis=0),
                 color='b',alpha=0.25)
plt.xlim([0.2,1.5])
leg = plt.legend(fontsize=24)
leg.get_frame().set_linewidth(0.0)
plt.yticks([])
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
plt.ylabel('')
plt.show()


X1 = np.array(XX1)[:3,t,:]
X2 = np.array(XX1)[3,t,:]
X3 = np.array(XX1)[-1,t,:]
T = []
for i in range(X1.shape[-2]):
    P1 = [x/sum(x) for x in X1[:,i,:]]
    T.append(np.array([sum(p[p>0]*np.log2(1/p[p>0])) for p in P1]).mean(axis=0))
    
T2_s = []
for x in X2:
    P1 = x/sum(x)
    T2_s.append([sum(p[p>0]*np.log2(1/p[p>0])) for p in [P1]])

T2_e = []
for x in X3:
    P1 = x/sum(x)
    T2_e.append([sum(p[p>0]*np.log2(1/p[p>0])) for p in [P1]])
        

In [ ]:
means = np.mean(S_MC.mean(axis=-1),axis=0)
sem   = sp.stats.sem(S_MC[0],axis=1)
plt.errorbar(x = np.arange(3,8),y = means[3:], yerr=sem[3:],lw=4,c='b')
plt.errorbar(x = np.arange(3),y = means[:3], yerr=sem[:3],lw=4,c='r')
plt.plot([2,3],[means[2],means[3]],'b',lw=4)
plt.ylim([0.46,0.54])
plt.yticks(fontsize=48)
plt.xticks(fontsize=48)
plt.show()

### g

In [ ]:
S_MC = []
for k in range(1000):
    S_MC.append(ContLNStim(0.2,0.11,0.6,0.14589222947969693))
S_MC = np.array(S_MC)
XX1 = []
for i in range(S_MC.shape[-2]):
    X1 = [] 
    for i,s in enumerate(S_MC[:,i,:]):
        x = np.histogram(s,bins=15,density=True,range=(np.nanmin(Araw), np.nanmax(Araw)))
        X1.append(x[0]/(sum(x[0])*(x[1][1]-x[1][0])))
    X2 = (x[1][:-1]+x[1][1:])/2
    XX1.append(X1)
for i,s in enumerate(S_MC[0,:,:]):
    x = np.histogram(s,bins=15,density=True,range=(np.nanmin(s), np.nanmax(s)))
    if(i<3):
        sns.kdeplot(s,c='r',alpha=0.5)
    else:
        sns.kdeplot(s,c='b',alpha=0.5)
plt.plot(X2,np.vstack(XX1[0]+XX1[1]+XX1[2]).mean(axis=0),'r',label='Pre-stim.',lw=3)
plt.fill_between(X2, np.vstack(XX1[0]+XX1[1]+XX1[2]).mean(axis=0)-np.vstack(XX1[0]+XX1[1]+XX1[2]).std(axis=0),
                 np.vstack(XX1[0]+XX1[1]+XX1[2]).mean(axis=0)+np.vstack(XX1[0]+XX1[1]+XX1[2]).std(axis=0),
                 color='r',alpha=0.25)
plt.plot(X2,np.vstack(XX1[3]+XX1[4]+XX1[6]+XX1[4]+XX1[7]).mean(axis=0),'b',label='Post-stim.',lw=3)
plt.fill_between(X2, np.vstack(XX1[3]+XX1[4]+XX1[6]+XX1[4]+XX1[7]).mean(axis=0)-np.vstack(XX1[3]+XX1[4]+XX1[6]+XX1[4]+XX1[7]).std(axis=0),
                 np.vstack(XX1[3]+XX1[4]+XX1[6]+XX1[4]+XX1[7]).mean(axis=0)+np.vstack(XX1[3]+XX1[4]+XX1[6]+XX1[4]+XX1[7]).std(axis=0),
                 color='b',alpha=0.25)
plt.xlim([0.2,1.5])
leg = plt.legend(fontsize=24)
leg.get_frame().set_linewidth(0.0)
plt.yticks([])
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
plt.ylabel('')
plt.show()

X1 = np.array(XX1)[:3,t,:]
X2 = np.array(XX1)[3,t,:]
X3 = np.array(XX1)[-1,t,:]
T = []
for i in range(X1.shape[-2]):
    P1 = [x/sum(x) for x in X1[:,i,:]]
    T.append(np.array([sum(p[p>0]*np.log2(1/p[p>0])) for p in P1]).mean(axis=0))
    
T3_s = []
for x in X2:
    P1 = x/sum(x)
    T3_s.append([sum(p[p>0]*np.log2(1/p[p>0])) for p in [P1]])

T3_e = []
for x in X3:
    P1 = x/sum(x)
    T3_e.append([sum(p[p>0]*np.log2(1/p[p>0])) for p in [P1]])
        

In [ ]:
means = np.mean(S_MC.mean(axis=-1),axis=0)
sem   = sp.stats.sem(S_MC[0],axis=1)
plt.errorbar(x = np.arange(3,8),y = means[3:], yerr=sem[3:],lw=4,c='b')
plt.errorbar(x = np.arange(3),y = means[:3], yerr=sem[:3],lw=4,c='r')
plt.plot([2,3],[means[2],means[3]],'b',lw=4)
plt.ylim([0.46,0.54])
plt.yticks(fontsize=48)
plt.xticks(fontsize=48)
plt.show()

### h

In [ ]:
def ContLNStimSmall(theta,theta2,mu,mu_add=0):
    fmean = sp.interpolate.interp1d(np.arange(-5.0,5.0,0.15),np.arange(-5.0,5.0,0.15)*ml.slope+ml.intercept)
    fstd = sp.interpolate.interp1d(np.arange(-5.0,5.0,0.15),np.arange(-5.0,5.0,0.15)*sl.slope+sl.intercept)
    fmean2 = sp.interpolate.interp1d(np.arange(-5.0,5.0,0.15),
                                    np.arange(-5.0,5.0,0.15)*-0.33988898633439907+0.17344471005740245)
    fstd2 = sp.interpolate.interp1d(np.arange(-5.0,5.0,0.15),
                                    np.arange(-5.0,5.0,0.15)*0.12238578174002662+0.05526252834977198)


    S = [A1raw15.T[0]]
    for i in range(1,8):
        sMean = fmean(S[-1]) + S[-1]
        sStd  = fstd(S[-1])
        if(i==3):
            sMean[S[-1]<0.35] += 0.15
            mu   += mu_add
        lmean = np.log((sMean**2)/np.sqrt(sStd**2+sMean**2))
        lsig = np.sqrt(np.log((sStd/sMean)**2 + 1))
        yi = sp.stats.lognorm.rvs(lsig, loc=-S[-1], scale=np.exp(lmean), size=len(S[0]))
        nS = S[-1]+(yi)-theta2*(S[-1]-mu)
        if(i>1 and i!=3):
            nS += theta*(S[-2]-S[-1]) 
        S.append(nS)
    return np.array(S)

S_MC = []
for k in range(1000):
    S_MC.append(ContLNStimSmall(0.2,0.11,0.6,0.14589222947969693))
S_MC = np.array(S_MC)
XX1 = []
for i in range(S_MC.shape[-2]):
    X1 = [] 
    for i,s in enumerate(S_MC[:,i,:]):
        x = np.histogram(s,bins=15,density=True,range=(np.nanmin(Araw), np.nanmax(Araw)))
        X1.append(x[0]/(sum(x[0])*(x[1][1]-x[1][0])))
    X2 = (x[1][:-1]+x[1][1:])/2
    XX1.append(X1)
for i,s in enumerate(S_MC[0,:,:]):
    x = np.histogram(s,bins=15,density=True,range=(np.nanmin(s), np.nanmax(s)))
    if(i<3):
        sns.kdeplot(s,c='r',alpha=0.5)
    else:
        sns.kdeplot(s,c='b',alpha=0.5)
plt.plot(X2,np.vstack(XX1[0]+XX1[1]+XX1[2]).mean(axis=0),'r',label='Pre-stim.',lw=3)
plt.fill_between(X2, np.vstack(XX1[0]+XX1[1]+XX1[2]).mean(axis=0)-np.vstack(XX1[0]+XX1[1]+XX1[2]).std(axis=0),
                 np.vstack(XX1[0]+XX1[1]+XX1[2]).mean(axis=0)+np.vstack(XX1[0]+XX1[1]+XX1[2]).std(axis=0),
                 color='r',alpha=0.25)
plt.plot(X2,np.vstack(XX1[3]+XX1[4]+XX1[6]+XX1[4]+XX1[7]).mean(axis=0),'b',label='Post-stim.',lw=3)
plt.fill_between(X2, np.vstack(XX1[3]+XX1[4]+XX1[6]+XX1[4]+XX1[7]).mean(axis=0)-np.vstack(XX1[3]+XX1[4]+XX1[6]+XX1[4]+XX1[7]).std(axis=0),
                 np.vstack(XX1[3]+XX1[4]+XX1[6]+XX1[4]+XX1[7]).mean(axis=0)+np.vstack(XX1[3]+XX1[4]+XX1[6]+XX1[4]+XX1[7]).std(axis=0),
                 color='b',alpha=0.25)
plt.xlim([0.2,1.5])
leg = plt.legend(fontsize=24)
leg.get_frame().set_linewidth(0.0)
plt.yticks([])
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
plt.ylabel('')
plt.show()


X1 = np.array(XX1)[:3,t,:]
X2 = np.array(XX1)[3,t,:]
X3 = np.array(XX1)[-1,t,:]
T = []
for i in range(X1.shape[-2]):
    P1 = [x/sum(x) for x in X1[:,i,:]]
    T.append(np.array([sum(p[p>0]*np.log2(1/p[p>0])) for p in P1]).mean(axis=0))
    
T4_s = []
for x in X2:
    P1 = x/sum(x)
    T4_s.append([sum(p[p>0]*np.log2(1/p[p>0])) for p in [P1]])

T4_e = []
for x in X3:
    P1 = x/sum(x)
    T4_e.append([sum(p[p>0]*np.log2(1/p[p>0])) for p in [P1]])
        

In [ ]:
means = np.mean(S_MC.mean(axis=-1),axis=0)
sem   = sp.stats.sem(S_MC[0],axis=1)
plt.errorbar(x = np.arange(3,8),y = means[3:], yerr=sem[3:],lw=4,c='b')
plt.errorbar(x = np.arange(3),y = means[:3], yerr=sem[:3],lw=4,c='r')
plt.plot([2,3],[means[2],means[3]],'b',lw=4)
plt.ylim([0.46,0.54])
plt.yticks(fontsize=48)
plt.xticks(fontsize=48)
plt.show()

### i

In [ ]:
X1 = np.array(XX1[:3]).mean(axis=0)
X2 = np.array(XX1[-1])
t = np.random.choice(1000,500)
P1 = np.array([x/sum(x) for x in X1])
P2_4 = np.array([x/sum(x) for x in X2])


#P1 = XX1[0].mean(axis=0)/sum(XX1[0].mean(axis=0))
#P2 = XX1[3].mean(axis=0)/sum(XX1[3].mean(axis=0))
boxprops = dict(linewidth=3, color='b')
whiskerprops = dict(linewidth=3, color='b')
capprops = dict(linewidth=3, color='b')
medianprops = dict( linewidth=3, color='r')
c='r'
plt.axhline(2.69,c='gray',lw=3,label='Expt. entropy \n after stim.')
y1 = np.array([2.75]*10)
y2 = np.array([2.67]*10)
plt.fill_between(np.linspace(-1.47500000000000003, 9.5,10),y2,y1,alpha=0.33,color='gray')
plt.boxplot(np.array(T),positions=[0],
            bootstrap=1000,widths=0.8,boxprops=dict(linewidth=3, color=c),
            capprops=dict(linewidth=3,color=c),
            whiskerprops=dict(linewidth=3,color=c),
            flierprops=dict(linewidth=3,color=c, markeredgecolor=c),
            medianprops=dict(linewidth=3,color='k'),labels=['pre-stim'],showfliers=False,)
c='b'
plt.boxplot(np.array(T1_e),positions=[3],
            bootstrap=1000,widths=0.8,boxprops=dict(linewidth=3, color=c),
            capprops=dict(linewidth=3,color=c),
            whiskerprops=dict(linewidth=3,color=c),
            flierprops=dict(linewidth=3,color=c, markeredgecolor=c),
            medianprops=dict(linewidth=3,color='k'),showfliers=False,labels=[r'C1'])
plt.boxplot(np.array(T2_e),positions=[5],
            bootstrap=1000,widths=0.8,boxprops=dict(linewidth=3, color=c),
            capprops=dict(linewidth=3,color=c),
            whiskerprops=dict(linewidth=3,color=c),
            flierprops=dict(linewidth=3,color=c, markeredgecolor=c),
            medianprops=dict(linewidth=3,color='k'),showfliers=False,labels=[r'C2'])

plt.boxplot(np.array(T3_e),positions=[7],
            bootstrap=1000,widths=0.8,boxprops=dict(linewidth=3, color=c),
            capprops=dict(linewidth=3,color=c),
            whiskerprops=dict(linewidth=3,color=c),
            flierprops=dict(linewidth=3,color=c, markeredgecolor=c),
            medianprops=dict(linewidth=3,color='k'),showfliers=False,labels=[r'C3'])

plt.boxplot(np.array(T4_e),positions=[9],
            bootstrap=1000,widths=0.8,boxprops=dict(linewidth=3, color=c),
            capprops=dict(linewidth=3,color=c),
            whiskerprops=dict(linewidth=3,color=c),
            flierprops=dict(linewidth=3,color=c, markeredgecolor=c),
            medianprops=dict(linewidth=3,color='k'),showfliers=False,labels=[r'C4'])

h = barplot_annotate_brackets(0,1,stats.ttest_ind(np.array(T)
                                                  ,np.array(T1_e)).pvalue,
                               [0,6], [2.43,2.86],fs=32)
h = barplot_annotate_brackets(0,1,stats.ttest_ind(np.array(T4_e)[t]
                                                  ,np.array(T3_e)[t]).pvalue,
                               [7,9], [2.88,2.88],fs=32)
ax = plt.gca()
plt.xticks(fontsize=32)
plt.gca().yaxis.tick_left()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
ax.set_xlim(-1.47500000000000003, 9.5)
plt.plot([3,9],[2.88,2.88],'k-')
plt.show()